In [1]:
import requests
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpBinary

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_players = pd.read_parquet('df_players.parquet')
df_positions = pd.read_parquet('df_positions.parquet')
df_teams = pd.read_parquet('df_teams.parquet')

In [2]:
# Supposons que df_players est votre DataFrame préparé
# Il doit contenir les colonnes : 'position', 'price', 'points_per_cost', 'minutes', 'selected_by_percent'

# Exemple initialisation ou import réel:
# df_players = pd.read_csv('votre_fichier.csv')
# df_players['selected_by_percent'] = df_players['selected_by_percent'].astype(float)

budget = 1500  # Exemple budget total (en 0.1M £)
alpha = 0    # Pondération ownership (à ajuster: plus grand = plus pénalisant)

df_players = df_players.reset_index(drop=True)  # Indices alignés

# Créer problème d'optimisation
prob = LpProblem("FPL_Team_Optimizer", LpMaximize)

# Variables binaires pour chaque joueur
player_vars = [LpVariable(f"player_{i}", cat=LpBinary) for i in range(len(df_players))]

# Fonction objectif : maximiser points_per_cost tout en minimisant selected_by_percent
prob += lpSum([
    df_players.loc[i, 'points_per_cost'] * player_vars[i] - 
    alpha * df_players.loc[i, 'selected_by_percent'] * player_vars[i]
    for i in range(len(df_players))
])

# Contraintes d’effectif
prob += lpSum(player_vars) == 15
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'GKP']) == 2
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'DEF']) == 5
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'MID']) == 5
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'FWD']) == 3

# Contrainte : max 3 joueurs par équipe
for club in df_players['team_name'].unique():
    prob += lpSum([
        player_vars[i]
        for i in range(len(df_players))
        if df_players.loc[i, 'team_name'] == club
    ]) <= 3

# Contraintes budget
prob += lpSum([df_players.loc[i, 'price'] * player_vars[i] for i in range(len(df_players))]) <= budget

# Contrainte minimum minutes (exemple)
prob += lpSum([df_players.loc[i, 'minutes'] * player_vars[i] for i in range(len(df_players))]) >= 1000

# Résoudre
prob.solve()

# Extraire liste des joueurs sélectionnés
selected_players = [df_players.loc[i] for i in range(len(df_players)) if player_vars[i].varValue == 1]
selected_df = pd.DataFrame(selected_players)

# Afficher résultats clés
print(selected_df[['web_name', 'position', 'price', 'points_per_cost', 'minutes', 'selected_by_percent']])


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/python/3.12.1/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/b2fff225dad84929b854736008654668-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/b2fff225dad84929b854736008654668-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 32 COLUMNS
At line 5313 RHS
At line 5341 BOUNDS
At line 6083 ENDATA
Problem MODEL has 27 rows, 741 columns and 3384 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 10.2018 - 0.00 seconds
Cgl0004I processed model has 27 rows, 622 columns (622 integer (552 of which binary)) and 2908 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 2.22045e-16
Cbc0038I Solution found of -10.2018
Cbc0038I Cleaned solution of -10.2018
Cbc0038I Before mini branch and bound, 622 integers at bound fixed and 0 continu

In [3]:
selected_df.sort_values(by = ['position'])

,first_name,web_name,team_short,team_name,position,price,total_points,points_per_cost,points_per_minutes,fdr_next_6,ict_index,selected_by_percent,selected_rank,form,minutes,transfers_in,transfers_in_event,transfers_out,transfers_out_event,assists,goals_scored,goal_involvements,expected_goals,expected_assists,expected_goal_involvements,GI_on_xGI,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,defensive_contribution_per_90,minutes_last,xGI_last,xG_last,ict_last,total_points_last,total_points_last_per_xGI_last
0,Marc,Guéhi,CRY,Crystal Palace,DEF,47.0,43.0,0.914894,0.079630,3.333333,18.0,26.0,10,8.8,540.0,1960065,59557,411426,1634,2.0,1.0,3.0,0.39,0.09,0.48,6.250000,0.06,0.00,0.02,0.08,1.14,0.50,7.83,90.000000,0.080000,0.065000,3.000000,7.166667,89.583333
1,Omar,Alderete,SUN,Sunderland,DEF,40.0,36.0,0.900000,0.073922,3.000000,19.8,2.6,112,7.8,487.0,244821,14742,68150,756,1.0,1.0,2.0,0.58,0.33,0.91,2.197802,0.11,0.00,0.06,0.17,1.21,0.74,10.90,81.166667,0.151667,0.096667,3.300000,6.000000,39.560440
2,Marcos,Senesi,BOU,Bournemouth,DEF,48.0,43.0,0.895833,0.079630,3.166667,20.2,18.6,19,8.0,540.0,2182063,59852,174233,1477,2.0,0.0,2.0,0.14,0.37,0.51,3.921569,0.02,0.00,0.06,0.08,0.91,1.17,13.50,90.000000,0.085000,0.023333,3.366667,7.166667,84.313725
4,Joe,Rodon,LEE,Leeds,DEF,40.0,29.0,0.725000,0.053704,2.666667,10.5,4.7,76,5.0,540.0,440513,15680,148596,1041,0.0,1.0,1.0,0.33,0.01,0.34,2.941176,0.06,0.00,0.00,0.06,1.22,1.50,9.33,90.000000,0.056667,0.055000,1.750000,4.833333,85.294118
6,Tyrick,Mitchell,CRY,Crystal Palace,DEF,50.0,34.0,0.680000,0.062963,3.333333,27.9,2.0,130,6.0,540.0,144282,6574,75765,683,0.0,1.0,1.0,0.28,0.46,0.74,1.351351,0.05,0.00,0.08,0.13,1.14,0.50,8.50,90.000000,0.123333,0.046667,4.650000,5.666667,45.945946
40,Wilson,Isidor,SUN,Sunderland,FWD,55.0,27.0,0.490909,0.084639,3.000000,22.7,2.7,109,5.2,319.0,351277,11377,82050,3030,0.0,3.0,3.0,1.23,0.03,1.26,2.380952,0.35,0.00,0.01,0.36,1.37,0.85,3.67,53.166667,0.210000,0.205000,3.783333,4.500000,21.428571
44,João Pedro,João Pedro,CHE,Chelsea,FWD,78.0,37.0,0.474359,0.072549,2.833333,36.3,67.7,1,5.0,510.0,2666790,12330,1079199,56755,3.0,2.0,5.0,1.50,0.23,1.73,2.890173,0.26,0.00,0.04,0.30,1.27,1.24,4.94,85.000000,0.288333,0.250000,6.050000,6.166667,21.387283
51,Richarlison,Richarlison,TOT,Spurs,FWD,68.0,31.0,0.455882,0.070938,2.666667,38.0,17.6,20,3.5,437.0,2222579,18929,814795,12412,1.0,3.0,4.0,1.66,0.19,1.85,2.162162,0.34,0.00,0.04,0.38,1.27,0.82,4.94,72.833333,0.308333,0.276667,6.333333,5.166667,16.756757
3,Robin,Roefs,SUN,Sunderland,GKP,45.0,37.0,0.822222,0.068519,3.000000,13.0,2.7,108,7.0,540.0,315483,8169,88918,954,0.0,0.0,0.0,0.00,0.01,0.01,0.000000,0.00,3.33,0.00,0.00,1.15,0.67,0.00,90.000000,0.001667,0.000000,2.166667,6.166667,3700.000000
11,Nick,Pope,NEW,Newcastle,GKP,50.0,32.0,0.640000,0.071111,3.000000,8.8,6.9,56,4.8,450.0,499500,16024,195452,1722,1.0,0.0,1.0,0.00,0.00,0.00,0.000000,0.00,2.00,0.00,0.00,0.53,0.60,0.00,75.000000,0.000000,0.000000,1.466667,5.333333,0.000000


In [4]:
selected_df['price'].sum()

np.float64(810.0)

In [5]:
selected_df['selected_by_percent'].sum()

np.float64(223.49999999999997)